In [10]:
import pandas as pd
import numpy as np
import os

from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical


raw_data_path = 'raw_data/'

def create_model():
    model = models.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['Emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'Pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


def data_to_tf_data(df):
    image_array, image_label = prepare_data(df)
    images = image_array.reshape((image_array.shape[0], 48, 48, 1))
    images = images.astype('float32')/255
    labels = to_categorical(image_label)
    return images, labels


In [11]:
train = pd.read_csv(raw_data_path+'initial_training_data.csv')
train_images, train_labels = data_to_tf_data(train)

val = pd.read_csv(raw_data_path+'validation_test_data.csv')
val_images, val_labels = data_to_tf_data(val)


model = create_model()
class_weight = dict(zip(range(0, 7), (((train['Emotion'].value_counts()).sort_index())/len(train['Emotion'])).tolist()))
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

df = pd.read_csv(raw_data_path+'test_data.csv')
test_images, test_labels = data_to_tf_data(df)
test_loss, test_acc = model.evaluate(test_images, test_labels)


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/12
6/6 [==============================] - 2s 254ms/step - loss: 0.3299 - accuracy: 0.2586 - val_loss: 1.9917 - val_accuracy: 0.2449
Epoch 2/12
6/6 [==============================] - 1s 212ms/step - loss: 0.3110 - accuracy: 0.2835 - val_loss: 1.8542 - val_accuracy: 0.2449
Epoch 3/12
6/6 [==============================] - 1s 212ms/step - loss: 0.3238 - accuracy: 0.2835 - val_loss: 1.8531 - val_accuracy: 0.2449
Epoch 4/12
6/6 [==============================] - 1s 210ms/step - loss: 0.3056 - accuracy: 0.2835 - val_loss: 2.0688 - val_accuracy: 0.2449
Epoch 5/12
6/6 [==============================] - 1s 212ms/step - loss: 0.3013 - accuracy: 0.2835 - val_loss: 2.0192 - val_accuracy: 0.2449
Epoch 6/12
6/6 [==============================] - 1s 211ms/step - loss: 0.2988 - accuracy: 0.2835 - val_loss: 2.0279 - val_accuracy: 0.2449
Epoch 7/12
6/6 [==============================] - 1s 209ms/step - loss: 0.2947 - accuracy: 0.2835 - val_loss: 1.9224 - val_accuracy: 0.2449
Epoch 8/12
6/6 [====

In [12]:
from tensorflow import keras

model.save("initial_model")


INFO:tensorflow:Assets written to: initial_model/assets


In [13]:
import os
from mlflow import log_metric, log_param, log_artifacts
import mlflow


In [14]:
mlflow.create_experiment('FinalProj2_initial')
mlflow.set_experiment('FinalProj2_initial')

In [15]:
!git init

Initialized empty Git repository in /home/ubuntu/notebooks/JacobD/JD_FinalProj2/.git/


In [16]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [17]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvc/plots/confusion.json
	new file:   .dvc/plots/confusion_normalized.json
	new file:   .dvc/plots/default.json
	new file:   .dvc/plots/linear.json
	new file:   .dvc/plots/scatter.json
	new file:   .dvc/plots/smooth.json
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	FinalProj2_Initial.ipynb
	initial_model/
	mlruns/
	raw_data/
	train_model.py
	train_model_2.py

